In [335]:
import networkx as nx
import random
from ortools.linear_solver import pywraplp

# Configurações gerais
n = 100  # número de vértices
p = 0.1  # probabilidade de aresta
random.seed(42)

# Gerando o grafo aleatório
G = nx.gnp_random_graph(n, p)
for (u, v) in G.edges():
    G.edges[u, v]['weight'] = random.randint(10, 500)

# Visualização básica (opcional)
print("Número de vértices:", G.number_of_nodes())
print("Número de arestas:", G.number_of_edges())


Número de vértices: 100
Número de arestas: 474


In [336]:
import time

def combinations(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i + 1, r):
            indices[j] = indices[j - 1] + 1
        yield tuple(pool[i] for i in indices)

def solve_with_subtour_elimination(G, d):
    start_time = time.time()
    # Criando o solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    # Variáveis de decisão: x[u, v] = 1 se a aresta (u, v) é selecionada, 0 caso contrário
    x = {}
    for u, v in G.edges():
        x[u, v] = solver.BoolVar(f'x_{u}_{v}')
        x[v, u] = solver.BoolVar(f'x_{v}_{u}')  # Considerando grafo não direcionado

    # Função objetivo: minimizar o custo das arestas selecionadas
    solver.Minimize(solver.Sum(G.edges[u, v]['weight'] * x[u, v] for u, v in G.edges()))

    # 1. Restrição de árvore geradora no subgrafo dos centrais: 
    centrais = d.keys()
    solver.Add(solver.Sum(x[u, v] for u, v in G.edges() if u in centrais and v in centrais) == len(centrais) - 1)

    # 2. Eliminação de ciclos (Subtour Elimination Constraints - SECs) no subgrafo dos centrais
    for r in range(2, len(centrais)):  # De tamanho 2 até tamanho len(centrais) - 1
        for subset in combinations(centrais, r):
            solver.Add(solver.Sum(x[u, v] for u, v in combinations(subset, 2) if G.has_edge(u, v)) <= len(subset) - 1)

    # 3. Restrição de grau mínimo para os vértices centrais
    for v in centrais:
        solver.Add(solver.Sum(x[u, v] for u in G.neighbors(v)) >= d[v])

    # 4. Cada terminal deve ser conectado a apenas um vértice central (grau 1)
    terminals = set(G.nodes()) - set(centrais)  # Supondo que vértices não em 'd' são terminais
    for t in terminals:
        solver.Add(solver.Sum(x[t, v] for v in G.neighbors(t)) == 1)

    # 5. Garantir que as variáveis são binárias (0 ou 1)
    for u, v in G.edges():
        solver.Add(x[u, v] == x[v, u])  # Impor que seja binário

    # Limite de tempo de execução (opcional)
    solver.SetTimeLimit(1800 * 1000)  # 1800 segundos

    # Resolução do problema
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print('Solução ótima encontrada!')
        return time.time() - start_time
    else:
        print('Nenhuma solução ótima encontrada.')
        return None


In [337]:
import time

def solve_vertex_labeling(grafo, centrais):
    start_time = time.time()
    
    # Converte o grafo não direcionado em um grafo direcionado
    grafo_direcionado = nx.DiGraph()
    for i, j, data in grafo.edges(data=True):
        grafo_direcionado.add_edge(i, j, weight=data.get('weight', 1))
        grafo_direcionado.add_edge(j, i, weight=data.get('weight', 1))

    # Inicializa o solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    # Cria as variáveis binárias y_ij = 1 se a aresta (i, j) estiver na solução, 0 caso contrário
    y = {}
    for i, j in grafo_direcionado.edges():
        y[i, j] = solver.BoolVar(f'y_{i}_{j}')  # Agora estamos considerando o grafo como direcionado

    # Cria as variáveis de rótulo u_i para cada vértice i
    u = {}
    for v in grafo_direcionado.nodes():
        u[v] = solver.IntVar(1, len(centrais) - 1, f'u_{v}')

    # Define o vértice raiz (primeiro central)
    r = list(centrais.keys())[0]
    u[r] = solver.IntVar(0, 0, f'u_{r}')  # Rótulo da raiz é 0

    # Função objetivo: minimizar o custo da árvore geradora
    objective = solver.Objective()
    for i, j, data in grafo_direcionado.edges(data=True):
        weight = data.get('weight', 1)  # Usa o peso da aresta, ou 1 se não houver peso
        objective.SetCoefficient(y[i, j], weight)
    objective.SetMinimization()

    # Restrição (6): Apenas um arco de entrada para cada vértice (exceto a raiz)
    for v in grafo_direcionado.nodes():
        if v != r:
            solver.Add(solver.Sum([y[i, v] for i in grafo_direcionado.predecessors(v)]) == 1)

    # Restrição (7) e (8): Grau mínimo dos vértices centrais
    for i in centrais:
        if i == r:
            solver.Add(solver.Sum([y[i, j] for j in grafo_direcionado.successors(i)]) >= centrais[i])
        else:
            solver.Add(solver.Sum([y[i, j] for j in grafo_direcionado.successors(i)]) >= centrais[i] - 1)

    # Restrição (9): Evitar ciclos (Miller-Tucker-Zemlin)
    for i, j in grafo_direcionado.edges():
        if i != r and j != r:
            solver.Add(u[i] - u[j] + len(centrais) * y[i, j] <= len(centrais) - 1)

    # Resolver o problema
    status = solver.Solve()

    # Verificar se a solução foi encontrada
    if status == pywraplp.Solver.OPTIMAL:
        print('Solução ótima encontrada!')
    else:
        print('Nenhuma solução ótima encontrada.')

    return time.time() - start_time


In [338]:
import networkx as nx

def read_graph_from_file(file_path):
    with open(file_path, 'r') as f:
        # Primeira linha: n (vértices), nc (centrais), m (arestas)
        n, nc, m = map(int, f.readline().strip().split())
        
        # Próximas nc linhas: centrais e graus mínimos
        centrais = {}
        for _ in range(nc):
            i, d = map(int, f.readline().strip().split())
            centrais[i] = d
        
        # Próximas m linhas: arestas e custos
        G = nx.Graph()
        for _ in range(m):
            i, j, c = map(int, f.readline().strip().split())
            G.add_edge(i, j, weight=c)
        
    return G, centrais

# Exemplo de uso
file_path = './Instancias/tb8ch15_0.txt'
grafo, centrais = read_graph_from_file(file_path)

# Exibindo o grafo e as informações dos vértices centrais
print("Grafo:", grafo.edges(data=True))
print("Centrais e graus mínimos:", centrais)


Grafo: [(1, 2, {'weight': 396}), (1, 3, {'weight': 283}), (1, 4, {'weight': 198}), (1, 5, {'weight': 256}), (1, 6, {'weight': 268}), (1, 7, {'weight': 260}), (1, 8, {'weight': 290}), (1, 9, {'weight': 228}), (1, 10, {'weight': 109}), (1, 11, {'weight': 257}), (1, 12, {'weight': 168}), (1, 13, {'weight': 170}), (1, 14, {'weight': 271}), (1, 15, {'weight': 343}), (1, 16, {'weight': 113}), (1, 17, {'weight': 37}), (1, 18, {'weight': 233}), (1, 19, {'weight': 299}), (1, 20, {'weight': 324}), (1, 21, {'weight': 351}), (1, 22, {'weight': 257}), (1, 23, {'weight': 126}), (1, 24, {'weight': 302}), (1, 25, {'weight': 255}), (1, 26, {'weight': 101}), (1, 27, {'weight': 172}), (1, 28, {'weight': 198}), (1, 29, {'weight': 179}), (1, 30, {'weight': 282}), (1, 31, {'weight': 184}), (1, 32, {'weight': 23}), (1, 33, {'weight': 286}), (1, 34, {'weight': 150}), (1, 35, {'weight': 316}), (1, 36, {'weight': 248}), (1, 37, {'weight': 305}), (1, 38, {'weight': 122}), (2, 3, {'weight': 676}), (2, 4, {'weight

In [339]:
import time

def compare_formulations(G,d):
    # Suponha que 'd' é um dicionário que você já definiu contendo os graus mínimos dos vértices centrais

    obj_value_subtour = solve_with_subtour_elimination(G, d)

    print(f"Subtour Elimination: Time = {obj_value_subtour} seconds")

    obj_value_mtz = solve_vertex_labeling(G, d)

    print(f"MTZ: Time = {obj_value_mtz} seconds")

# Exemplo de comparação
compare_formulations(grafo, centrais)


Solução ótima encontrada!
Subtour Elimination: Time = 6.531088829040527 seconds
Solução ótima encontrada!
MTZ: Time = 71.81130576133728 seconds
